In [1]:
import osmnet
import json
import networkx as nx
from shapely.geometry import Point, shape
from scipy import spatial
import numpy as np
import pickle
import pandas as pd

In [2]:
def rename_nodes(nodes_df, edges_df, node_id_name, to_name, from_name):
    nodes_df['old_node_id']=nodes_df[node_id_name].copy()
    nodes_df['node_id']=range(len(nodes_df))
    node_name_map={nodes_df.iloc[i]['old_node_id']: i for i in range(len(nodes_df))}
    edges_df['from_node_id']=edges_df.apply(lambda row: node_name_map[row[from_name]], axis=1)
    edges_df['to_node_id']=edges_df.apply(lambda row: node_name_map[row[to_name]], axis=1)
    return nodes_df, edges_df, node_name_map

In [3]:
city='Detroit'

ALL_ZONES_PATH='./clean/model_area.geojson'
PORTALS_PATH='./clean/portals.geojson'
TABLE_AREA_PATH='./clean/table_area.geojson'

# networks from CS_Accessibility- placed in folder manually for now
PT_NODES_PATH='./clean/comb_network_nodes.csv'
PT_EDGES_PATH='./clean/comb_network_edges.csv'
PED_NODES_PATH='./clean/osm_ped_network_nodes.csv'
PED_EDGES_PATH='./clean/osm_ped_network_edges.csv'


ROUTE_COSTS_PATH='./clean/route_costs.json'
SIM_GRAPHS_PATH='./clean/sim_area_nets.p'
SIM_NET_GEOJSON_PATH='./clean/'

SPEEDS_MET_S={'driving':30/3.6,
        'cycling':15/3.6,
        'walking':4.8/3.6}

pandana_link_types={'osm to transit': 'waiting',
                    'transit to osm': 'waiting',
                    'walk': 'walking',
                    'transit': 'pt'
                    }


In [4]:
# =============================================================================
# Load network data
# =============================================================================
# get the area bounds
all_zones_shp=json.load(open(ALL_ZONES_PATH))
if city=='Hamburg':
    all_zones_geoid_order=[f['properties']['GEO_ID'] for f in all_zones_shp['features']]
else:
    all_zones_geoid_order=[f['properties']['GEO_ID'].split('US')[1] for f in all_zones_shp['features']]

portals=json.load(open(PORTALS_PATH))
table_area=json.load(open(TABLE_AREA_PATH))




In [5]:
min_lon, min_lat, max_lon, max_lat=float('inf'), float('inf'), -float('inf'), -float('inf')
for f in all_zones_shp['features']:
    for polygon in f['geometry']['coordinates']:
        for vertex in polygon:
            if vertex[0]<min_lon:
                min_lon=vertex[0]
            if vertex[0]>max_lon:
                max_lon=vertex[0]
            if vertex[1]<min_lat:
                min_lat=vertex[1]
            if vertex[1]>max_lat:
                max_lat=vertex[1]
boundsAll=[min_lon,min_lat, max_lon, max_lat]

In [6]:
drive_nodes,drive_edges=osmnet.load.network_from_bbox(lat_min=boundsAll[1], lng_min=boundsAll[0], lat_max=boundsAll[3], 
                              lng_max=boundsAll[2], bbox=None, network_type='drive', 
                              two_way=True, timeout=180, 
                              custom_osm_filter=None)

Requesting network data within bounding box from Overpass API in 4 request(s)
Posting to http://www.overpass-api.de/api/interpreter with timeout=180, "{'data': '[out:json][timeout:180];(way["highway"]["highway"!~"cycleway|footway|path|pedestrian|steps|track|proposed|construction|bridleway|abandoned|platform|raceway|service"]["motor_vehicle"!~"no"]["motorcar"!~"no"]["service"!~"parking|parking_aisle|driveway|emergency_access"](42.03147965,-83.21226437,42.24793831,-82.87033451);>;);out;'}"
Downloaded 4,407.1KB from www.overpass-api.de in 0.75 seconds
Posting to http://www.overpass-api.de/api/interpreter with timeout=180, "{'data': '[out:json][timeout:180];(way["highway"]["highway"!~"cycleway|footway|path|pedestrian|steps|track|proposed|construction|bridleway|abandoned|platform|raceway|service"]["motor_vehicle"!~"no"]["motorcar"!~"no"]["service"!~"parking|parking_aisle|driveway|emergency_access"](42.03147965,-83.55190700,42.24185042,-83.20496860);>;);out;'}"
Downloaded 7,409.1KB from www.

In [7]:
# get the osm as nodes and edges dfs
# TODO: try one-way and find routes between highest degree node in each zone
cycle_nodes,cycle_edges= drive_nodes.copy(),drive_edges.copy()

# get the pt net as nodes and edges dfs
pt_edges=pd.read_csv(PT_EDGES_PATH)
pt_nodes=pd.read_csv(PT_NODES_PATH)

walk_edges=pd.read_csv(PED_EDGES_PATH)
walk_nodes=pd.read_csv(PED_NODES_PATH)

# renumber nodes in both networks as 1 to N
pt_nodes, pt_edges, _ =rename_nodes(pt_nodes, pt_edges, 'id_int', 'to_int', 'from_int')
drive_nodes, drive_edges, _=rename_nodes(drive_nodes, drive_edges, 'id', 'to', 'from')
walk_nodes, walk_edges, _=rename_nodes(walk_nodes, walk_edges, 'id', 'to', 'from')
cycle_nodes, cycle_edges, _=rename_nodes(cycle_nodes, cycle_edges, 'id', 'to', 'from')

network_dfs={'driving': {'edges':drive_edges, 'nodes': drive_nodes} ,
              'pt': {'edges':pt_edges, 'nodes': pt_nodes},
              'walking': {'edges':walk_edges, 'nodes': walk_nodes},
              'cycling': {'edges':cycle_edges, 'nodes': cycle_nodes}}

/Users/doorleyr/anaconda/envs/urbanaccess/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (1,2,3,5,6,7,8,9,10,11,12,14,15,18,19,20,21,22,23,24) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/Users/doorleyr/anaconda/envs/urbanaccess/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (1,6,8,9) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/Users/doorleyr/anaconda/envs/urbanaccess/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (1,8,13,15,16) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [8]:
for osm_mode in ['driving', 'walking', 'cycling']:
    G=nx.Graph()
    for i, row in network_dfs[osm_mode]['edges'].iterrows():
        G.add_edge(row['from_node_id'], row['to_node_id'], attr_dict={
                'weight_minutes':(row['distance']/SPEEDS_MET_S[osm_mode])/60,
                'type': osm_mode})
    network_dfs[osm_mode]['graph']=G
    
G_pt=nx.Graph()
for i, row in network_dfs['pt']['edges'].iterrows():
    G_pt.add_edge(row['from_node_id'], row['to_node_id'], 
                     attr_dict={'weight_minutes':row['weight'],
                                'type': pandana_link_types[row['net_type']]})
network_dfs['pt']['graph']=G_pt

In [9]:
#for each network
#for each portal
#find nodes inside the portal and add zero_cost links to/from those
for net in network_dfs:
    for p in range(len(portals['features'])):
        p_shape=shape(portals['features'][p]['geometry'])
        nodes_inside=[n for n in range(len(network_dfs[net]['nodes'])) if p_shape.contains(
                Point([network_dfs[net]['nodes'].iloc[n]['x'],
                       network_dfs[net]['nodes'].iloc[n]['y']]))]
        for ni in nodes_inside:
            network_dfs[net]['graph'].add_edge('p'+str(p), ni,
                       attr_dict={'type': 'from_portal', 'weight_minutes':0, 'distance': 0})
            network_dfs[net]['graph'].add_edge(ni, 'p'+str(p),
                       attr_dict={'type': 'to_portal', 'weight_minutes':0, 'distance': 0})

#import matplotlib.pyplot as plt
#colors={'driving': 'red', 'pt': 'green'}
#plt.figure()
## Test: find the nodes that connect to each portal
neighbours={}
for net in network_dfs:
    neighbours[net]={}
    for p in range(len(portals['features'])):
        if network_dfs[net]['graph'].has_node('p'+str(p)):
            nb=[n for n in network_dfs[net]['graph'].neighbors('p'+str(p))]
            neighbours[net][p]=nb 
        else:
            neighbours[net][p]=[]
#xs=[network_dfs[net]['nodes'].iloc[n]['x'] for n in neighbours[net][0]]+[network_dfs[net]['nodes'].iloc[n]['x'] for n in neighbours[net][1]] +    [network_dfs[net]['nodes'].iloc[n]['x'] for n in neighbours[net][2]] + [network_dfs[net]['nodes'].iloc[n]['x'] for n in neighbours[net][3]] 
#ys=[network_dfs[net]['nodes'].iloc[n]['y'] for n in neighbours[net][0]]+[network_dfs[net]['nodes'].iloc[n]['y'] for n in neighbours[net][1]] +    [network_dfs[net]['nodes'].iloc[n]['y'] for n in neighbours[net][2]] + [network_dfs[net]['nodes'].iloc[n]['y'] for n in neighbours[net][3]] 
#plt.scatter(xs, ys)

# =============================================================================
# Find routes
# =============================================================================
## get the N closest nodes to the centre of each zone
lon_lat_list= [[shape(f['geometry']).centroid.x, shape(f['geometry']).centroid.y
                ] for f in all_zones_shp['features']]  
closest_nodes={}
for net in network_dfs:
    closest_nodes[net]=[]
    kdtree_nodes=spatial.KDTree(np.array(network_dfs[net]['nodes'][['x', 'y']]))
    for i in range(len(lon_lat_list)):
        _, c_nodes=kdtree_nodes.query(lon_lat_list[i], 10)
        closest_nodes[net].append(list(c_nodes))

In [10]:
from ipyleaflet import Map, Polyline, basemaps, basemap_to_tiles, Circle, Marker
%matplotlib inline

In [11]:
colors=['#1b9e77',
'#d95f02',
'#7570b3',
'#e7298a',
'#66a61e']

In [12]:
polylines=[]
mode='driving'
zone=200
routes=[]
m = Map(center = (42.35, -83.2), zoom =12, layers=[basemap_to_tiles(basemaps.CartoDB.DarkMatter)])
for p in range(len(portals['features'])):
    node_path=nx.shortest_path(network_dfs[mode]['graph'],closest_nodes[mode][zone][0], 
                                                  'p'+str(p), weight='weight')
    node_coords=[list(network_dfs[mode]['nodes'].iloc[n][['y', 'x']]) for n in node_path[:-1]]
    line = Polyline(
        locations = [node_coords],
        weight=2,
        opacity=1,
        color = colors[p],
        fill=False
    )
    m.add_layer(line)
m    

Map(basemap={'url': 'https://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png', 'max_zoom': 19, 'attribution': 'Map …

In [16]:
polylines=[]
routes=[]
destination=[-83.065940, 42.332349]
dest_node=kdtree_nodes.query(destination, 1)[1]
m = Map(center = (42.35, -83.2), zoom =12, layers=[basemap_to_tiles(basemaps.CartoDB.DarkMatter)])
for p in range(len(portals['features'])):
    node_path=nx.shortest_path(network_dfs[mode]['graph'], 
                                                  'p'+str(p), dest_node, weight='weight')
    node_coords=[list(network_dfs[mode]['nodes'].iloc[n][['y', 'x']]) for n in node_path[1:]]
    line = Polyline(
        locations = [node_coords],
        weight=2,
        opacity=1,
        color = colors[p],
        fill=False
    )
    m.add_layer(line)
m  

Map(basemap={'url': 'https://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png', 'max_zoom': 19, 'attribution': 'Map …

Park and Autonomus Ride Example

In [35]:
polylines=[]
routes=[]
destinations=[[  -83.066988, 42.331442], [ -83.064231, 42.327925,]]
origin_portals=[0,1]
m = Map(center = (42.326623, -83.073996), zoom =14, layers=[basemap_to_tiles(basemaps.CartoDB.DarkMatter)])
for i in range(len(origin_portals)):
    dest_node=kdtree_nodes.query(destinations[i], 1)[1]
    node_path=nx.shortest_path(network_dfs[mode]['graph'], 
                                                  'p'+str(origin_portals[i]), dest_node, weight='weight')
    node_coords=[list(network_dfs[mode]['nodes'].iloc[n][['y', 'x']]) for n in node_path[1:]]
    line = Polyline(
        locations = [node_coords],
        weight=2,
        opacity=0.5,
        color = 'red',
        fill=False
    )
    m.add_layer(line)
m  

Map(basemap={'url': 'https://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png', 'max_zoom': 19, 'attribution': 'Map …

In [36]:
polylines=[]
routes=[]
dest_nodes=[]
parking_locations=[ -83.082386, 42.329627]
m = Map(center = (42.326623, -83.073996), zoom =14, layers=[basemap_to_tiles(basemaps.CartoDB.DarkMatter)])
for i in range(len(origin_portals)):
    park_node=kdtree_nodes.query(parking_location, 1)[1]
    dest_nodes.append(kdtree_nodes.query(destinations[i], 1)[1])
    node_path_to_parking=nx.shortest_path(network_dfs[mode]['graph'], 
                                                  'p'+str(origin_portals[i]), park_node, weight='weight')
    node_coords=[list(network_dfs[mode]['nodes'].iloc[n][['y', 'x']]) for n in node_path_to_parking[1:]]
    line = Polyline(
        locations = [node_coords],
        weight=2,
        opacity=0.5,
        color = 'red',
        fill=False
    )
    m.add_layer(line)
# shared ride from parking to destinations
node_path_to_dest=nx.shortest_path(network_dfs[mode]['graph'], 
                                              park_node, dest_nodes[0], weight='weight')
node_path_to_dest+=nx.shortest_path(network_dfs[mode]['graph'], 
                                              dest_nodes[0], dest_nodes[1], weight='weight')
node_coords=[list(network_dfs[mode]['nodes'].iloc[n][['y', 'x']]) for n in node_path_to_dest]
line = Polyline(
    locations = [node_coords],
    weight=2,
    opacity=0.5,
    color = 'yellow',
    fill=False
)
m.add_layer(line) 
m  

Map(basemap={'url': 'https://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png', 'max_zoom': 19, 'attribution': 'Map …